<a href="https://colab.research.google.com/github/Mikalaj-Plck/Data_parsing/blob/main/HTML_Beautiful_soup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import requests
from bs4 import BeautifulSoup
import json
import time
import re

def clean_price(price_str):
    # Очистка строки цены от лишних символов и кодировок
    # Удаляем специальные символы и пробелы
    price_str = re.sub(r'[^\d.,]', '', price_str)

    # Заменяем запятую на точку, если используется запятая
    price_str = price_str.replace(',', '.')

    try:
        return float(price_str)
    except ValueError:
        print(f"Не удалось преобразовать цену: {price_str}")
        return 0.0

def scrape_books():
    base_url = "http://books.toscrape.com/catalogue/"
    index_url = "http://books.toscrape.com/index.html"
    all_books = []

    # Получаем список категорий
    try:
        response = requests.get(index_url)
        response.encoding = 'utf-8'  # Явно устанавливаем кодировку
        soup = BeautifulSoup(response.text, 'html.parser')

        # Находим ссылки на категории
        category_links = [
            "http://books.toscrape.com/" + link['href']
            for link in soup.select('ul.nav-list ul li a')
        ]
    except Exception as e:
        print(f"Ошибка при получении списка категорий: {e}")
        return []

    # Обработка каждой категории
    for category_url in category_links:
        page_num = 1
        while True:
            try:
                # Первая страница категории
                if page_num == 1:
                    current_url = category_url
                else:
                    # Для последующих страниц
                    current_url = category_url.replace('index.html', f'page-{page_num}.html')

                # Загрузка страницы
                response = requests.get(current_url)
                response.encoding = 'utf-8'  # Явно устанавливаем кодировку

                # Если страница не найдена, переходим к следующей категории
                if response.status_code != 200:
                    break

                soup = BeautifulSoup(response.text, 'html.parser')

                # Находим все книги на странице
                books = soup.select('article.product_pod')

                # Если книг нет, прекращаем обработку категории
                if not books:
                    break

                # Обработка каждой книги
                for book in books:
                    try:
                        # Получаем ссылку на книгу
                        book_relative_link = book.select_one('h3 a')['href']
                        book_link = base_url + book_relative_link.replace('../', '')

                        # Загружаем страницу книги
                        book_response = requests.get(book_link)
                        book_response.encoding = 'utf-8'  # Явно устанавливаем кодировку
                        book_soup = BeautifulSoup(book_response.text, 'html.parser')

                        # Извлекаем информацию
                        title = book.select_one('h3 a')['title']

                        # Очистка цены с использованием специальной функции
                        price_elem = book.select_one('p.price_color')
                        price = clean_price(price_elem.text) if price_elem else 0.0

                        # Количество в наличии
                        stock_text = book.select_one('p.instock').text.strip()
                        stock_match = re.search(r'\((\d+) available\)', stock_text)
                        stock = int(stock_match.group(1)) if stock_match else 0

                        # Описание
                        description_elem = book_soup.select_one('#product_description + p')
                        description = description_elem.text.strip() if description_elem else "Нет описания"

                        # Сохраняем информацию о книге
                        book_info = {
                            "title": title,
                            "price": price,
                            "stock": stock,
                            "description": description
                        }

                        all_books.append(book_info)

                    except Exception as book_error:
                        print(f"Ошибка при обработке книги: {book_error}")

                # Переход к следующей странице
                page_num += 1

                # Небольшая задержка
                time.sleep(0.5)

            except Exception as category_error:
                print(f"Ошибка при обработке категории: {category_error}")
                break

    # Сохраняем результаты
    with open('books_data.json', 'w', encoding='utf-8') as f:
        json.dump(all_books, f, ensure_ascii=False, indent=4)

    print(f"Скрейпинг завершен. Всего книг: {len(all_books)}")
    return all_books

# Запуск парсера
if __name__ == "__main__":
    scrape_books()

Скрейпинг завершен. Всего книг: 1000
